# Creating Clean TON_IoT Data
Source: https://research.unsw.edu.au/projects/toniot-datasets (See TON_IoT datasets/Processed_datasets/Processed_Network_dataset)
____

## README

**This Python notebook is only for generating data from the raw datasets.**


The pre-generated file (team_11_TON_IoT_unsw_edu_au_cleaned.csv), is already available in Step1-Datasets-Feature-Engineering.

If you simply want to use the data, use the team11_TON_IoT data.

If you wish to change the data cleaning parameters, this is the right place.


In [ ]:
import pandas as pd
from pathlib import Path
current_working_directory = Path.cwd()

## Read datasets

When exporting is complete, please delete the Processed_Network_dataset folder. 

In [ ]:
# process data
ds_array = []
for i in range(1, 24):
    try:
        ds_array.append(pd.read_csv(f"./raw/Processed_Network_dataset/Network_dataset_{i}.csv"))
    except FileNotFoundError as e:
        print(f"To run this notebook, extract the and move the 'Processed_Network_dataset' folder of Processed Network dataset.zip to '{current_working_directory}/raw'.")
        print("The notebook will now raise an error.")
        raise e

# Analyzing data

## What's included.

In [ ]:
# quick check to see available types
what_i_use = []
for i in range(0, 23):
    for e in ds_array[i]["type"].unique():
        if e not in what_i_use:
            what_i_use.append(e)
print(f"What the data finds: {what_i_use}")

In [ ]:
# print counts per type
for i in range(0, 23):
    print(f"Spreadsheet {i+1}") 
    print(ds_array[i]["type"].value_counts(), end="\n\n")

# Data cleaning

## Removing unneccessary columns

In [ ]:
# cleaning
for i in range(0, 23):
    ds_array[i] = ds_array[i].drop(columns=[
        "service","dns_query", "dns_AA", 
        "dns_RD", "dns_RA", "dns_rejected", 
        "ssl_version", "ssl_cipher", "ssl_resumed", 
        "ssl_established", "ssl_subject", "ssl_issuer", 
        "http_trans_depth", "http_method", "http_uri", 
        "http_version", "http_request_body_len", "http_response_body_len", 
        "http_status_code", "http_user_agent", "http_orig_mime_types", 
        "http_resp_mime_types", "weird_name", "weird_addl", 
        "weird_notice", "dns_qclass",
        "dns_qtype", "dns_rcode", "http_referrer", ])
    if "uid" in ds_array[i]:
        ds_array[i] = ds_array[i].drop(columns=["uid"])

## Drop specific attack types: 
* password
* xss
* ransomware
* backdoor
* injection
* mitm

In [ ]:
# drop non-network based attacks
for i in range(0, 23):
    ds_array[i]= ds_array[i][ds_array[i]["type"] != "password"]
    ds_array[i]= ds_array[i][ds_array[i]["type"] != "xss"]
    ds_array[i]= ds_array[i][ds_array[i]["type"] != "ransomware"]
    ds_array[i]= ds_array[i][ds_array[i]["type"] != "backdoor"]
    ds_array[i]= ds_array[i][ds_array[i]["type"] != "injection"]
    ds_array[i]= ds_array[i][ds_array[i]["type"] != "mitm"]

### If you wish to see the individual cleaned CSV files:
change the following line to True.

In [ ]:
EXPORT_INDIVIDUAL_CSV = False

In [ ]:
# see above
if EXPORT_INDIVIDUAL_CSV == True:
    for i in range(0, 23):
        ds_array[i].to_csv(f"./team_11_individual_TON_IoT_unsw_edu_au_{i+1}.csv")

## Reduce dataset

In [ ]:
# merge data
newDS  = pd.concat(ds_array, ignore_index=True)

### Correct data error with src_bytes
For some packets, src_bytes becomes 0.0.0.0, which is fixed by setting to 0.

In [ ]:
ip_error = newDS[newDS['src_bytes'] == "0.0.0.0"]
newDS.loc[ip_error.index,"src_bytes"] = 0

### Drop unneeded scanning entries
Decided by taking a fraction close to, but less than the amount of 'normal' entries.

In [ ]:
SCANNING_ENTRIES = 649106
D_DOS_ENTRIES = 474231
DOS_ENTRIES = 421916

### Drop unneeded scanning entries

In [ ]:
temp = newDS[newDS["type"] == "scanning"]
temp = temp.reset_index()
# from the end of the data capture and up
newDS = newDS.drop(index=list(temp[-(len(temp)- SCANNING_ENTRIES):]["index"]))

### Drop unneeded DDoS entries

In [ ]:
temp = newDS[newDS["type"] == "ddos"]
temp = temp.reset_index()
# from the end of the data capture and up
newDS = newDS.drop(index=list(temp[-(len(temp)- D_DOS_ENTRIES):]["index"]))

### Drop unneeded DoS entries

In [ ]:
temp = newDS[newDS["type"] == "dos"]
temp = temp.reset_index()
# from the end of the data capture and up
newDS = newDS.drop(index=list(temp[-(len(temp)- DOS_ENTRIES):]["index"]))

In [ ]:
newDS["type"].value_counts()

# Export

In [ ]:
newDS.to_csv("../Step1-Datasets-Feature-Engineering/team11_TON_IoT_unsw_edu_au_cleaned.csv", index=None)